In [1]:
## required packages
import numpy as np
# import matplotlib.pyplot as plt
# import os, time
import importlib
import im.fcn_im as fcn_im
import pandas as pd
# from numpy.random import standard_normal
# from scipy.linalg import cholesky
import model, fcn_gen, os, time
from scipy import sparse
# import OpenSHAInterface as sha

In [2]:
# base inputs
site_loc_file = r'C:\Users\barry\Desktop\OpenSHAInterface\CA_pipe_midpt_allsites.txt'
rate_min = 1/10000
im_tool = 'sha'

im_dir = r'C:\Users\barry\Desktop\OpenSHAInterface\im_output_10000yr_allsites'
dm_dir = r'C:\Users\barry\Desktop\OpenSHAInterface\rr_out'
rup_meta_file = r'C:\Users\barry\Desktop\OpenSHAInterface\rup_meta_10000yr_allsites.hdf5'
# sha_meta_file = r'C:\Users\barry\Desktop\OpenSHAInterface\sha_meta_10000yr.txt'
groups2run = np.loadtxt(r'C:\Users\barry\Desktop\OpenSHAInterface\groups2run_allsites.txt',dtype=str)
group_num = 0
num_rups = 2
im_list = ['pgv']

## define number of realizations and generate samples
nsamp=10
flag_corr_d = False
flag_corr_T = False
# geo_cond = 2 #  for spatial correlation

len(groups2run)

1977

In [3]:
## reload model.py if it has been modified
importlib.reload(model)
## create class object
A = model.assessment()

In [4]:
count = 0
inc1 = 5
inc2 = 100
# out = {}
time_start = time.time()

In [5]:
## for generating samples
multi = 17
range_start = 0+100*multi
range_end = range_start+100
# range_end = range_start+1
flag_sample_exist = True
for im_group in groups2run[range_start:range_end]:
    #####################################################################
    count += 1
    path_sample = im_dir+'/'+im_group+'/'
    
    #####################################################################
    ## load GM predictions and create random variable
    A.create_RV(im_tool, site_loc_file, im_dir, im_list, rup_meta_file, flag_clear_dict=True, 
                flag_sample_exist=flag_sample_exist, im_group=im_group, num_rups=num_rups)

    #####################################################################
    ## generate/import samples
    A.get_IM(nsamp, im_list, flag_corr_d, flag_corr_T, flag_clear_dict=True, 
            flag_sample_exist=flag_sample_exist,path_sample=path_sample)

    if flag_sample_exist is False:
        for i in range(nsamp):
            out1 = sparse.csc_matrix(np.round(A._IM_dict['pgv'][i].log1p(),decimals=3))
            out2 = sparse.csc_matrix(np.round(A._IM_dict['pga'][i].log1p(),decimals=3))
            sparse.save_npz(im_dir+'/'+im_group+'/'+'pgv_samp_'+str(i)+'.npz',out1)
            sparse.save_npz(im_dir+'/'+im_group+'/'+'pga_samp_'+str(i)+'.npz',out2)

    #####################################################################
    dm_method = []
    ## get rr from HAZUS (FEMA, 2014)
    category = 'rr'
    method = 'hazus_2014_rr'
    return_param = ['rr_pgv']
    flag_pgv = True
    flag_pga = False
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category, method, return_param, im_list, flag_clear_dict=False,
             flag_pga=flag_pga, flag_pgv=flag_pgv, nsamp=nsamp)
#     print(A._DM_dict['rr_pgv']['method1']['output'][0][0:5])
    
    ## get rr from ALA (2001)
    category = 'rr'
    method = 'orourke_2020_rr'
    return_param = ['rr_pgv']
    flag_pgv = True
    flag_pga = False
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category, method, return_param, im_list, flag_clear_dict=False,
             flag_pga=flag_pga, flag_pgv=flag_pgv, nsamp=nsamp)
#     print(A._DM_dict['rr_pgv']['method2']['output'][0][0:5])

    #####################################################################
#     if count % inc1 == 0:
    print('folder '+str(count)+': '+im_group+'--- %10.6f seconds ---' % (time.time() - time_start))
    time_start = time.time()

    #####################################################################
    if count % inc2 == 0 or count == len(groups2run)-1:
        print('saved files: '+str(range_start)+'_'+str(range_end))
        for i in range(len(return_param)):
            count_dm = 0
            for j in A._DM_dict[return_param[i]]:
                save_name = dm_dir+'/'+return_param[i]+'_'+dm_method[count_dm]+'_'+str(range_start)+'_'+str(range_end)+'.txt'
#                 print(save_name)
                np.savetxt(save_name,np.transpose(A._DM_dict[return_param[i]][j]['output']),fmt='%6.4E')
                count_dm += 1

folder 1: 74200_74299---  20.545357 seconds ---
folder 2: 74300_74399---   7.421833 seconds ---
folder 3: 74400_74499---   6.812997 seconds ---
folder 4: 74500_74599---   6.116251 seconds ---
folder 5: 74600_74699---   4.291358 seconds ---
folder 6: 74700_74799---   4.525334 seconds ---
folder 7: 74800_74899---   5.180621 seconds ---
folder 8: 74900_74999---   5.652496 seconds ---
folder 9: 75000_75099---   7.958999 seconds ---
folder 10: 7500_7599---  14.510593 seconds ---
folder 11: 75100_75199---  11.702639 seconds ---
folder 12: 75200_75299---  14.602524 seconds ---
folder 13: 75300_75399---  16.808194 seconds ---
folder 14: 75400_75499---  17.557511 seconds ---
folder 15: 75500_75599---  21.960078 seconds ---
folder 16: 75600_75699---  18.750677 seconds ---
folder 17: 75700_75799---  17.579216 seconds ---
folder 18: 75800_75899---  14.470861 seconds ---
folder 19: 75900_75999---  12.641633 seconds ---
folder 20: 76000_76099---  10.636208 seconds ---
folder 21: 7600_7699---  15.843

In [29]:
dm_method

['hazus', 'ala']